# Agentic RAG System

## Load Azure Configuration

In [66]:
from dotenv import load_dotenv
import os

load_dotenv() # take environment variables from .env.

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_embeddings_deployment = os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT")
azure_openai_api_version = "2024-10-01-preview"
azure_openai_embedding_size = 1536

azure_cosmosdb_endpoint = os.getenv("AZURE_COSMOSDB_ENDPOINT")
azure_cosmosdb_key = os.getenv("AZURE_COSMOSDB_KEY")

azure_search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
azure_search_service_admin_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")



## Function Definitions

In [68]:
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from typing import Annotated
from openai import AzureOpenAI
from azure.cosmos import CosmosClient
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential


class RAGPlugin:
    """A plugin that gets data from multiple sources."""

    @kernel_function(description="Provides information about recipes and cooking")
    def get_recipes(self, 
        user_query: Annotated[str, "The user query"]) -> Annotated[str, "Returns the recipes or cooking information."]:
        
        # Setup the connection
        azure_cosmosdb_database = "recipes-database"
        azure_cosmosdb_container = "recipes-container"
        cosmos_client = CosmosClient(url=azure_cosmosdb_endpoint, credential=azure_cosmosdb_key)
        database = cosmos_client.get_database_client(azure_cosmosdb_database)
        container = database.get_container_client(azure_cosmosdb_container)

        # Azure OpenAI client
        openai_client = AzureOpenAI(
        api_version=azure_openai_api_version,
        azure_endpoint=azure_openai_endpoint,
        azure_deployment=azure_openai_embeddings_deployment,
        api_key=azure_openai_key)

        # get embedding of user query
        response = openai_client.embeddings.create(input=user_query, 
                                                model=azure_openai_embeddings_deployment, 
                                                dimensions=azure_openai_embedding_size)
        embedding = response.data[0].embedding

        # format the query
        query ='''
                    SELECT TOP {0} 
                        c.id, 
                        c.name,
                        c.description,
                        c.cuisine,
                        c.difficulty,
                        c.prepTime,
                        c.cookTime,
                        c.totalTime,
                        c.servings,
                        c.ingredients,
                        c.instructions, 
                        VectorDistance(c.contentVector,{1}) AS SimilarityScore 
                    FROM c 
                    ORDER BY RANK RRF 
                        (VectorDistance(c.contentVector, {1}), FullTextScore(c.description, ['{2}']))
                '''.format(5, embedding, user_query)
        
        results = container.query_items(
                query=query,
                enable_cross_partition_query=True)
        
        items = [item for item in results]

        return items
    
    @kernel_function(description="Provides information about tech related stuff like Azure")
    def get_azure_services(self, 
        user_query: Annotated[str, "The user query"]) -> Annotated[str, "Returns the Azure services or tech related information."]:
        
        # Setup the connection
        azure_cosmosdb_database = "azureservicesdatabase01"
        azure_cosmosdb_container = "azureservicescontainer01"
        cosmos_client = CosmosClient(url=azure_cosmosdb_endpoint, credential=azure_cosmosdb_key)
        database = cosmos_client.get_database_client(azure_cosmosdb_database)
        container = container = database.get_container_client(azure_cosmosdb_container)

        # Azure OpenAI client
        openai_client = AzureOpenAI(
        api_version=azure_openai_api_version,
        azure_endpoint=azure_openai_endpoint,
        azure_deployment=azure_openai_embeddings_deployment,
        api_key=azure_openai_key)

        response = openai_client.embeddings.create(input=user_query, 
                                                model=azure_openai_embeddings_deployment, 
                                                dimensions=1536)
        embedding = response.data[0].embedding


        # Build the query with str.format() method
        query = '''
            SELECT TOP {0} c.id, c.title, c.category, c.content
            FROM c
            ORDER BY RANK RRF 
                (VectorDistance(c.contentVector, {1}), FullTextScore(c.title, ['{2}']))
        '''.format(5, embedding, user_query)

        results = container.query_items(
                query=query,
                enable_cross_partition_query=True)
        
        items = [item for item in results]

        return items
    
    @kernel_function(description="Provides information about NASA related stuff and geography")
    def get_geography_information(self, 
        user_query: Annotated[str, "The user query"]) -> Annotated[str, "Returns the information about NASA or geography."]:
        
        # index name
        azure_search_service_index_name = "ai-search-index-001"

        # User Query
        query = user_query

        # Get credential from Azure AI Search Admin key
        credential = AzureKeyCredential(azure_search_service_admin_key)
        search_client = SearchClient(endpoint=azure_search_service_endpoint, 
                                    credential=credential, 
                                    index_name=azure_search_service_index_name)

        # Convert query into vector form
        vector_query = VectorizableTextQuery(text=query, 
                                            k_nearest_neighbors=50, 
                                            fields="text_vector",
                                            weight=1)

        results = search_client.search(
            query_type="semantic", 
            semantic_configuration_name='my-semantic-config',
            search_text=query,
            vector_queries= [vector_query],
            select=["title","chunk","locations"],
            top=5,
        )

        sources_formatted = "=================\n".join([f'TITLE: {document["title"]}, CONTENT: {document["chunk"]}, LOCATIONS: {document["locations"]}' for document in results])
        return sources_formatted
    
    @kernel_function(description="Provides information travel related products")
    def get_travel_products(self, 
        user_query: Annotated[str, "The user query"]) -> Annotated[str, "Returns the information about travel related products."]:
        
        # index name
        azure_search_service_index_name = "product-index-challenge"

        # User Query
        query = user_query

        # Get credential from Azure AI Search Admin key
        credential = AzureKeyCredential(azure_search_service_admin_key)
        search_client = SearchClient(endpoint=azure_search_service_endpoint, 
                                    credential=credential, 
                                    index_name=azure_search_service_index_name)

        # Convert query into vector form
        vector_query = VectorizableTextQuery(text=query, 
                                            k_nearest_neighbors=50, 
                                            fields="text_vector",
                                            weight=1)

        results = search_client.search(
            query_type="semantic", 
            semantic_configuration_name='my-semantic-config',
            search_text=query,
            vector_queries= [vector_query],
            select=["title","chunk"],
            top=5,
        )

        # Use a unique separator to make the sources distinct. 
        # We chose repeated equal signs (=) followed by a newline because it's unlikely the source documents contain this sequence.
        sources_formatted = "=================\n".join([f'TITLE: {document["title"]}, CONTENT: {document["chunk"]}' for document in results])

        return sources_formatted


## Agent Definition and setup

In [76]:
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.kernel import Kernel
from semantic_kernel.functions import KernelArguments

# Create the instance of the Kernel
kernel = Kernel()

# Add the AzureChatCompletion AI Service to the Kernel
azure_service_id = "azure"
kernel.add_service(AzureChatCompletion(service_id=azure_service_id))

settings = kernel.get_prompt_execution_settings_from_service_id(service_id=azure_service_id)
# Configure the function choice behavior to auto invoke kernel functions
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Add the plugin to the kernel
kernel.add_plugin(RAGPlugin(), plugin_name="RAGPlugin")

# Create the agent
agent = ChatCompletionAgent(
    kernel=kernel, 
    name="RAGAgent", 
    instructions="""
        Answer questions about information related to the datasources you have access to.
        You will always provide the source of the information by displaying the name or title of the document source.
    """, 
    arguments=KernelArguments(
        settings=settings,
    ),
)

# Start by creating a ChatHistoryAgentThread object to maintain the conversation state
thread: ChatHistoryAgentThread = None


In [81]:
user_input = "What is the prep time for Bruschetta and how do you prepare it?"
async for response in agent.invoke(messages=user_input, thread=thread):
    print(f"{response.content}")
    thread = response.thread


The **prep time for Bruschetta** is approximately 15 minutes.

### How to Prepare Bruschetta

#### Ingredients:
- 4 slices of crusty Italian bread
- 2 ripe tomatoes, diced
- 1/4 cup fresh basil leaves, chopped
- 2 cloves garlic, minced
- 2 tablespoons extra-virgin olive oil
- 1 tablespoon balsamic vinegar
- Salt, to taste
- Black pepper, to taste

#### Instructions:
1. Preheat the oven to 375°F (190°C).
2. Place the slices of bread on a baking sheet and drizzle them with olive oil. Toast in the preheated oven for about 8-10 minutes, or until the bread is crispy and golden brown.
3. In a bowl, combine the diced tomatoes, minced garlic, chopped basil leaves, olive oil, and balsamic vinegar. Mix well to combine all the ingredients.
4. Season the tomato mixture with salt and black pepper according to taste. Stir well.
5. Remove the toasted bread from the oven and let it cool slightly. Rub each slice with a clove of garlic to infuse the bread with its flavor.
6. Spoon the tomato mixture gen

In [82]:
user_input = "What is Azure Firewall?"
async for response in agent.invoke(messages=user_input, thread=thread):
    print(f"{response.content}")
    thread = response.thread


**Azure Firewall** is a managed, cloud-based network security service designed to protect your Azure Virtual Network resources. 

### Key Features:
- **Stateful Packet Inspection**: Monitors the state and context of active connections, making it possible to filter traffic based on state, port, and protocol.
- **Application Filtering**: Allows for the control of outbound HTTP/S traffic based on categories and can prevent unauthorized applications from communicating.
- **Threat Intelligence**: Identifies and blocks known malicious IP addresses using threat intelligence feeds.
- **Support for Various Network Protocols**: Handles TCP, UDP, and ICMP traffic, making it versatile for different network scenarios.

### Integration and Use:
- **Network Security Policies**: Creates and enforces policies to manage and restrict network traffic, thus enhancing security.
- **Unauthorized Access Prevention**: Ensures that only authorized traffic is allowed through the network, protecting applications 

In [83]:
user_input = "What can you see in the Arctic Ocean?"
async for response in agent.invoke(messages=user_input, thread=thread):
    print(f"{response.content}")
    thread = response.thread

In the Arctic Ocean, you can witness several unique and fascinating phenomena:

### Ice Features:
- **Sea Ice**: Large expanses of floating ice cover much of the Arctic Ocean, with intricate patterns of wavy tendrils formed by newly created thin sea ice.
- **Cloud Streets**: These are parallel rows of clouds formed by winds blowing from the cold ice surface over the warmer, moister air near the open ocean. They are visible due to the spinning air cylinders that develop, creating areas of cloud formation and clear skies in a repeating pattern.

### Scenic Views:
- **Snow and Ice-Covered Landscapes**: The Arctic Ocean is surrounded by snow and ice, particularly in the easternmost reaches of Russia, contributing to its vast, white, frozen wilderness.

These features make the Arctic Ocean a visually captivating and scientifically significant region.

**Source**: Document page-21


In [84]:
user_input = "What is the price of the TrailMaster X4 Tent?"
async for response in agent.invoke(messages=user_input, thread=thread):
    print(f"{response.content}")
    thread = response.thread

The price of the **TrailMaster X4 Tent** is **$250**.

**Source**: Product Information Document
